In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
param_date = '20240706'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel(f'./{param_date}/9.url.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        for b in range(666):
            try:
                import requests

                import sys
                sys.path.append('../00.Tools')
                from crawler_configuration import get_header, get_proxy

                resp = requests.get(input_.loc[a, 'Url'],
                                    headers=get_header(),
                                    proxies=get_proxy(),
                                    timeout=(10, 10)).text

                # = = = = = = = = = = = = = = =

                from bs4 import BeautifulSoup
                from lxml import etree

                soup = BeautifulSoup(resp.strip(), 'lxml')
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =

                type_ = html.xpath('//h1/span[@id="lblPartTypeName"]/text()')[0]

                # = = = = = = = = = = = = = = =

                i, dict_src = 0, {}
                list_src = list(reversed([href.strip() for href in html.xpath('//a[@data-zoom-id="ProductPic"]/@href')]))
                for src in list_src:
                    if src.startswith('http'):
                        dict_src[str(i)] = src
                        i += 1

                # = = = = = = = = = = = = = = =

                part_number = html.xpath('//span[@id="lblProductName"]/text()')[0].strip()

                # = = = = = = = = = = = = = = =

                description = html.xpath('//span[@id="lblProductDesc"]/text()')[0].strip()

                # = = = = = = = = = = = = = = =

                application_summary = '\n'.join(html.xpath('//div[@id="divAppSummary"]/p/text()')).strip()

                # = = = = = = = = = = = = = = =

                brand = html.xpath('//span[@id="lblBrandName"]/text()')[0].strip()

                # = = = = = = = = = = = = = = =

                vehicle_url = html.xpath('//input[@id="hidfDetailApp"]/@value')
                vehicle_url = f'''https://www.dormanproducts.com/{vehicle_url[0].strip()}''' if vehicle_url else ''

                # = = = = = = = = = = = = = = =

                dict_specification = {}
                list_tr = html.xpath('//section[@id="productSpec"]/div/table/tr')
                for i, tr in zip(range(len(list_tr)), list_tr):
                    dict_specification[str(i)] = {tr.xpath('./th/text()')[0].strip().removesuffix(':').strip(): tr.xpath('./td/text()')[0].strip()}

                # = = = = = = = = = = = = = = =

                list_oem = [text.strip() for text in html.xpath('//section[@id="productOE"]/div/table/tbody/tr/th/text()')]
                oem = ';'.join(sorted(list(set(list_oem)), key=list_oem.index))

                # = = = = = = = = = = = = = = =

                import json

                df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                         'Part Number': part_number,
                                         'Type': type_,
                                         'Brand': brand,
                                         'Description': description,
                                         'Application Summary': application_summary,
                                         'Vehicle Url': vehicle_url,
                                         'OEM': oem,
                                         'Url': input_.loc[a, 'Url'],
                                         'Json_Src': json.dumps(dict_src),
                                         'Json_Specification': json.dumps(dict_specification)}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'

                # = = = = = = = = = = = = = = =

                break
            except:
                continue

        # = = = = = = = = = = = = = = =

        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                     'Url': input_.loc[a, 'Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b+1:3d}] - {input_.loc[a, 'No']}.{input_.loc[a, 'Url']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./{param_date}/10.part-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel(f'./{param_date}/part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[状态：ok，尝试次数：  1] - 96275.https://www.dormanproducts.com/p-9771-419-302.aspx
[剩余数量：1000] - [当前时间：21:16:43]

[状态：ok，尝试次数：  1] - 96277.https://www.dormanproducts.com/p-97711-h622400.aspx
[剩余数量：999] - [当前时间：21:16:43]

[状态：ok，尝试次数：  1] - 96272.https://www.dormanproducts.com/p-97701-h622485.aspx
[剩余数量：998] - [当前时间：21:16:44]

[状态：ok，尝试次数：  1] - 96274.https://www.dormanproducts.com/p-97709-h622370.aspx
[剩余数量：997] - [当前时间：21:16:44]

[状态：ok，尝试次数：  1] - 96283.https://www.dormanproducts.com/p-97720-h622486.aspx
[剩余数量：996] - [当前时间：21:16:45]

[状态：ok，尝试次数：  1] - 96282.https://www.dormanproducts.com/p-9772-419-607.aspx
[剩余数量：995] - [当前时间：21:16:45]

[状态：ok，尝试次数：  1] - 96288.https://www.dormanproducts.com/p-9773-419-613.aspx
[剩余数量：994] - [当前时间：21:16:46]

[状态：ok，尝试次数：  1] - 96281.https://www.dormanproducts.com/p-97719-h622371.aspx
[剩余数量：993] - [当前时间：21:16:47]

[状态：ok，尝试次数：  1] - 96287.https://www.dormanproducts.com/p-97729-h622377.aspx
[剩余数量：992] - [当前时间：21:16:47]

[状态：ok，尝试次数：  1] - 96286.https://www.do